In [1]:
import pandas as pd
import numpy as np 
from pathlib import Path
from warnings import simplefilter
simplefilter("ignore")  # ignore warnings to clean up output cells
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
#################### DATA ############################

data = pd.read_csv('data/data2022_2023.csv')
results = pd.read_csv('data/results.csv', index_col = 'resultId')
drivers = pd.read_csv('data/drivers.csv', index_col = 'driverId')
circuits = pd.read_csv('data/circuits.csv', index_col = 'circuitId')
races = pd.read_csv('data/races.csv', index_col = 'raceId')
stand = pd.read_csv('data/driver_standings.csv', index_col= 'driverStandingsId')
drivers.info()
circuits.info()
results.info()
results.head()
circuits.head()
drivers.head()
results.raceId.value_counts()
results[results.raceId == 800].T
drivers.loc[509]
# trying to forecast the future year ending results for HAMILTON

hami_results = results[results['driverId'] == 1].loc[:,['raceId','position', 'points', 'rank']]
hami_results  
hami_results.replace('\\N', np.nan, inplace=True)
np.where(hami_results.isna())[0]
hami_results = hami_results.dropna()
############################################################################################
########## Simple Predictions ###########
# predicting position
# Training data

'''X = hami_results.loc[:, ['raceId']]  # features
y = hami_results.loc[:, 'position']  # target
# Train the model
model = LinearRegression()
model.fit(X, y)
# Store the fitted values as a time series with the same time index as
# the training data
y_pred = pd.Series(model.predict(X), index=X.index)
ax = y_pred.plot()
ax.set_title('Hamiltons finishing positions through his races')'''

'''# predicting points
# Training data
X = hami_results.loc[:, ['raceId']]  # features
y = hami_results.loc[:, 'points']  # target
# Train the model
model = LinearRegression()
model.fit(X, y)
# Store the fitted values as a time series with the same time index as
# the training data
y_pred = pd.Series(model.predict(X), index=X.index)
ax = y_pred.plot()
ax.set_title('Hamiltons points through his races')'''
######### Lag Variables #########

'''hami_results['Lag_1'] = hami_results['points'].shift(1)
X = hami_results.loc[:, ['Lag_1']]
X.dropna(inplace=True)  # drop missing values in the feature set
y = hami_results.loc[:, 'points']  # create the target
y, X = y.align(X, join='inner')  # drop corresponding values in target
model = LinearRegression()
model.fit(X, y)
y_pred = pd.Series(model.predict(X), index=X.index)
fig, ax = plt.subplots()
ax.plot(X['Lag_1'], y, '.', color='0.25')
ax.plot(X['Lag_1'], y_pred)
ax.set_aspect('equal')
ax.set_ylabel('points')
ax.set_xlabel('Lag_1')
ax.set_title('Lag Plot of points scored by Hamilton')
ax = y_pred.plot()'''
##################### CNN ########################

from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

<class 'pandas.core.frame.DataFrame'>
Index: 857 entries, 1 to 858
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   driverRef    857 non-null    object
 1   number       857 non-null    object
 2   code         857 non-null    object
 3   forename     857 non-null    object
 4   surname      857 non-null    object
 5   dob          857 non-null    object
 6   nationality  857 non-null    object
 7   url          857 non-null    object
dtypes: object(8)
memory usage: 60.3+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 77 entries, 1 to 79
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   circuitRef  77 non-null     object 
 1   name        77 non-null     object 
 2   location    77 non-null     object 
 3   country     77 non-null     object 
 4   lat         77 non-null     float64
 5   lng         77 non-null     float64
 6   alt         77 non-n

In [2]:
hami_results
# Convert the dataset to float32

hami_results = hami_results.astype('float16')

In [3]:
selected_X_data = hami_results.iloc[:, [0, 2, 3]]

In [4]:

selected_y_data = hami_results.position

In [5]:


selected_X_data = selected_X_data.values.reshape(-1, 12, 2)  # Assuming n_steps=3 and n_features=1

In [6]:
selected_y_data = selected_y_data.values.reshape(-1, 2, 2)  # Assuming n_steps=3 and n_features=1

In [7]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(10, 3)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [8]:
X_train = selected_X_data  # Assuming selected_data is your input data

In [9]:
y_train = selected_y_data  # Assuming y_train is your target data

In [10]:
print(X_train.shape)

(37, 12, 2)


In [11]:
print(y_train.shape)

(74, 2, 2)


In [12]:
model.fit(X_train, y_train, epochs=10)

ValueError: Data cardinality is ambiguous:
  x sizes: 37
  y sizes: 74
Make sure all arrays contain the same number of samples.

In [13]:
selected_X_data = selected_X_data.values.reshape(-1, 2, 2)  # Assuming n_steps=3 and n_features=1

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [14]:
selected_X_data.values.shape

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [15]:
selected_X_data.shape

(37, 12, 2)

In [16]:
selected_X_data = hami_results.iloc[:, [0, 2, 3]]

In [17]:
selected_X_data.shape

(296, 3)

In [18]:
selected_y_data = hami_results.position

In [19]:
selected_y_data.shape

(296,)